In [6]:
import mesa
from epstein_network_civil_violence.agent import Inhabitant, Police
from epstein_network_civil_violence.model import EpsteinNetworkCivilViolence
import numpy as np
import matplotlib.pyplot as plt
from SALib.sample import saltelli
from SALib.analyze import sobol
import random

def GSA(problem, num_samples=2 ** 10):
    # 生成样本
    param_values = saltelli.sample(problem, num_samples)

    print("配置个数：", param_values.shape[0],end="   ")

    # 将视野值转换为整数，并筛选样本以确保citizen_density和cop_density的和不超过1
    param_values[:, 2] = np.round(param_values[:, 2]).astype(int)
    param_values[:, 3] = np.round(param_values[:, 3]).astype(int)

    # 调整citizen_density和cop_density的组合
    for i in range(len(param_values)):
        citizen_density = param_values[i, 0]
        cop_density = param_values[i, 1]
        if citizen_density + cop_density > 1:
            if random.random() > 0.5:
                param_values[i, 0] = 1 - cop_density
            else:
                param_values[i, 1] = 1 - citizen_density

    # filtered_param_values = param_values[np.sum(param_values[:, :2], axis=1) <= 1]

    # 评估模型
    Y = evaluate(param_values)

    # 计算敏感性指标
    Si = sobol.analyze(problem, Y, print_to_console=True)

    # 打印结果
    print('一阶敏感性指数:', Si['S1'])
    print('总敏感性指数:', Si['ST'])

# 定义模型函数
def evaluate(params):
    print("配置个数：", params.shape[0])
    results = []
    for i in range(params.shape[0]):
        # print(f"循环次数： {i}")
        # print(params[i])

        citizen_density = params[i, 0]
        cop_density = params[i, 1]
        citizen_vision = int(params[i, 2])
        cop_vision = int(params[i, 3])
        legitimacy = params[i, 4]
        active_threshold = params[i, 5]
        # print(cop_vision)

        model = EpsteinNetworkCivilViolence(
            width=40,
            height=40,
            citizen_density=citizen_density,
            cop_density=cop_density,
            citizen_vision=citizen_vision,
            cop_vision=cop_vision,
            legitimacy=legitimacy,
            max_jail_term=15,
            active_threshold=active_threshold,
            arrest_prob_constant=2.3,
            movement=True,
            max_iters=100010,
            alpha=0.1,
            jail_factor=1,
            impact_chance=0.5,
            legitimacy_impact=0.01,
            incitation_threshold=10
        )

        active_num = 0
        for _ in range(10):
            model.step()
            active_num += model.count_type_citizens(model, "Active")

        results.append(active_num)
        # print(f'Sample {i}: Active citizens = {active_num}')

    return np.array(results)




In [7]:
# 定义问题
problem = {
    'num_vars': 6,
    'names': ['citizen_density', 'cop_density', 'citizen_vision', 'cop_vision', 'legitimacy', 'active_threshold'],
    'bounds': [[0, 1], [0, 0.2], [1, 10], [1, 10], [0, 1], [0, 0.2]]
}

num_samples = 2**2

GSA(problem, num_samples=num_samples)

C:\Users\liaojw\AppData\Local\Temp\ipykernel_11100\1227014936.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, num_samples)


配置个数： 56   配置个数： 56
                        ST   ST_conf
citizen_density   0.002555  0.003657
cop_density       0.031079  0.044472
citizen_vision    0.463411  0.662494
cop_vision        0.005309  0.007597
legitimacy        0.000004  0.000005
active_threshold  0.000908  0.001300
                        S1   S1_conf
citizen_density  -0.078099  0.111755
cop_density       0.272367  0.389739
citizen_vision   -1.051758  2.185921
cop_vision       -0.112574  0.161086
legitimacy       -0.002941  0.004208
active_threshold -0.046565  0.066632
                                           S2    S2_conf
(citizen_density, cop_density)      -0.359964  74.880003
(citizen_density, citizen_vision)    0.176417  74.342407
(citizen_density, cop_vision)       -0.204027  74.737946
(citizen_density, legitimacy)       -0.248439  74.778364
(citizen_density, active_threshold) -0.230767  74.762277
(cop_density, citizen_vision)        0.017680  83.082419
(cop_density, cop_vision)           -1.110797  84.170696
(cop_d

In [8]:
# 定义问题
problem = {
    'num_vars': 6,
    'names': ['citizen_density', 'cop_density', 'citizen_vision', 'cop_vision', 'legitimacy', 'active_threshold'],
    'bounds': [[0, 1], [0, 0.2], [1, 10], [1, 10], [0, 1], [0, 0.2]]
}

num_samples = 2**4

GSA(problem, num_samples=num_samples)

C:\Users\liaojw\AppData\Local\Temp\ipykernel_11100\1227014936.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, num_samples)


配置个数： 224   配置个数： 224
Outburst ends at step 9
Outburst starts, wait time recorded: 10
Outburst ends at step 19
Outburst ends at step 7
                        ST   ST_conf
citizen_density   0.212060  0.256761
cop_density       0.479482  1.142307
citizen_vision    1.011362  2.283864
cop_vision        0.032460  0.023290
legitimacy        0.254182  0.660345
active_threshold  0.052186  0.085100
                        S1   S1_conf
citizen_density  -0.176693  0.533424
cop_density      -0.110425  0.646061
citizen_vision   -0.587263  0.769069
cop_vision       -0.007070  0.252711
legitimacy       -0.284533  0.416089
active_threshold  0.204641  0.277043
                                           S2   S2_conf
(citizen_density, cop_density)       1.116425  2.449001
(citizen_density, citizen_vision)    0.997544  3.630921
(citizen_density, cop_vision)        0.077096  0.884360
(citizen_density, legitimacy)        0.850627  1.794350
(citizen_density, active_threshold) -0.240708  0.770969
(cop_densit

In [9]:
# 定义问题
problem = {
    'num_vars': 6,
    'names': ['citizen_density', 'cop_density', 'citizen_vision', 'cop_vision', 'legitimacy', 'active_threshold'],
    'bounds': [[0, 1], [0, 0.2], [1, 10], [1, 10], [0, 1], [0, 0.2]]
}

num_samples = 2**6

GSA(problem, num_samples=num_samples)

C:\Users\liaojw\AppData\Local\Temp\ipykernel_11100\1227014936.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, num_samples)


配置个数： 896   配置个数： 896
Outburst ends at step 5
Outburst ends at step 7
Outburst starts, wait time recorded: 12
Outburst ends at step 13
Outburst starts, wait time recorded: 2
Outburst ends at step 7
Outburst ends at step 9
Outburst ends at step 9
Outburst ends at step 3
Outburst ends at step 5
Outburst starts, wait time recorded: 6
Outburst ends at step 13
Outburst starts, wait time recorded: 6
Outburst ends at step 15
Outburst ends at step 17
Outburst ends at step 7
Outburst starts, wait time recorded: 4
Outburst ends at step 13
Outburst starts, wait time recorded: 2
Outburst ends at step 19
Outburst ends at step 5
Outburst ends at step 11
Outburst ends at step 13
Outburst starts, wait time recorded: 4
Outburst ends at step 9
Outburst starts, wait time recorded: 4
Outburst ends at step 15
Outburst starts, wait time recorded: 2
Outburst ends at step 3
Outburst ends at step 5
Outburst starts, wait time recorded: 2
Outburst ends at step 13
                        ST   ST_conf
citizen_dens

In [13]:
# 定义问题
problem = {
    'num_vars': 6,
    'names': ['citizen_density', 'cop_density', 'citizen_vision', 'cop_vision', 'legitimacy', 'active_threshold'],
    'bounds': [[0, 1], [0, 0.2], [1, 10], [1, 10], [0, 1], [0, 0.2]]
}

num_samples = 2**6

GSA(problem, num_samples=num_samples)

C:\Users\liaojw\AppData\Local\Temp\ipykernel_11100\1227014936.py:12: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, num_samples)


配置个数： 896   配置个数： 896
Outburst ends at step 7
Outburst ends at step 9
Outburst starts, wait time recorded: 2
Outburst ends at step 13
Outburst ends at step 11
Outburst starts, wait time recorded: 6
Outburst ends at step 3
Outburst ends at step 7
Outburst starts, wait time recorded: 2
Outburst ends at step 11
Outburst ends at step 5
Outburst ends at step 9
Outburst ends at step 5
Outburst starts, wait time recorded: 2
Outburst ends at step 9
Outburst ends at step 5
Outburst ends at step 3
Outburst ends at step 5
Outburst ends at step 19
Outburst ends at step 5
Outburst ends at step 3
Outburst ends at step 15
Outburst ends at step 19
Outburst ends at step 15
Outburst ends at step 19
Outburst ends at step 5
Outburst ends at step 9
                        ST   ST_conf
citizen_density   0.671568  0.320170
cop_density       0.824568  0.514438
citizen_vision    0.887464  0.409719
cop_vision        0.098007  0.101326
legitimacy        0.045983  0.046011
active_threshold  0.135577  0.160805
   